In [1]:
pip install deep-translator

In [2]:
from deep_translator import GoogleTranslator
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from math import sqrt
from unidecode import unidecode
import os
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
...

Ellipsis

Configuracion Especial de Pandas

In [3]:
# Elimina los limites para mostrar columnas en Pandas
pd.set_option('display.max_columns', None)

## Comprensión del Problema

- Motivados a las necesidades de mantener una alimentación saludable, plantea realizar una aplicacion para recomendar Alimentos.  en este sentido se han recopilado 5 archivos .csv los cuales contienen alimentos con sus caracteristicas proteicas y de vitaminicas.
- Se plantea utilizar el algoritmo NearestNeighbours para clusterizar el dataset por similitudes y luego utilizarlo en la aplicación.

## Carga de datos
Se Cargan todos los archivos que se utilizaran y se compilan en un solo Archivo para iniciar el proceso EDA

In [4]:
# Lectura de archivos.csv
data_1 = pd.read_csv("C:\\Users\\HP\\Desktop\\FORMACION\\HackaBoss Data Scienct AI\\proyecto_III\\sources\\FOOD-BEBIDAS.csv")
data_2 = pd.read_csv("C:\\Users\\HP\\Desktop\\FORMACION\\HackaBoss Data Scienct AI\\proyecto_III\\sources\\FOOD-COMUNES.csv")
data_3 = pd.read_csv("C:\\Users\\HP\\Desktop\\FORMACION\\HackaBoss Data Scienct AI\\proyecto_III\\sources\\FOOD-FRUTAS.csv")
data_4 = pd.read_csv("C:\\Users\\HP\\Desktop\\FORMACION\\HackaBoss Data Scienct AI\\proyecto_III\\sources\\FOOD-PESCADOS.csv")
data_5 = pd.read_csv("C:\\Users\\HP\\Desktop\\FORMACION\\HackaBoss Data Scienct AI\\proyecto_III\\sources\\FOOD-POSTRES.csv")
#Compilacion de dataframes
data_compilada = pd.concat([data_1, data_2, data_3, data_4, data_5], ignore_index=True)
data_compilada

,Unnamed: 0.1,Unnamed: 0,food,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,Protein,Dietary Fiber,Cholesterol,Sodium,Water,Vitamin A,Vitamin B1,Vitamin B11,Vitamin B12,Vitamin B2,Vitamin B3,Vitamin B5,Vitamin B6,Vitamin C,Vitamin D,Vitamin E,Vitamin K,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
0,0,0,eggnog,224,10.6,6.600,3.300,0.500,20.4,20.400,11.600,0.0,149.9,0.100,209.7,0.1,0.074,0.020,0.006,0.500,0.3,1.100,0.100,3.8,0.055,0.500,0.011,330.200,0.051,0.500,48.3,0.024,276.90,419.1,0.094,1.200,377.200
1,1,1,beer light,96,0.0,0.000,0.000,0.000,5.4,0.300,0.800,0.0,0.0,0.095,313.1,0.0,0.083,0.079,0.000,0.069,1.3,0.067,0.100,0.0,0.000,0.000,0.000,13.200,0.095,0.014,16.5,0.094,39.60,69.3,0.077,0.044,19.456
2,2,2,beer budweiser,12,0.0,0.000,0.000,0.000,0.9,0.000,0.100,0.0,0.0,0.095,27.4,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,1.200,0.095,0.000,2.1,0.038,3.80,9.7,0.000,0.000,2.200
3,3,3,weizenbier erdinger,220,18.0,13.000,1.000,0.000,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.000,0.000,18.000
4,4,4,beer light budweiser,9,0.0,0.000,0.000,0.000,0.4,0.000,0.066,0.0,0.0,0.016,28.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.900,0.088,0.000,2.1,0.007,3.20,7.7,0.000,0.000,1.320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,227,227,muesli master crumble,124,2.4,1.000,19.800,8.500,4.5,0.042,2.400,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.000,0.000,9.300
2391,228,228,bran flakes,131,0.8,0.200,0.100,0.500,32.2,7.400,4.000,7.3,0.0,0.200,1.4,0.0,0.300,0.500,0.400,0.011,0.6,6.700,0.300,0.7,0.000,0.090,0.300,0.025,17.600,0.300,11.2,91.600,1.40,179.6,213.200,0.083,45.358
2392,229,229,nut cereal,245,5.9,1.000,3.900,0.500,46.2,17.200,4.000,3.6,0.0,0.300,2.6,0.2,0.500,0.300,0.000,0.600,6.5,0.300,0.700,10.7,0.002,0.000,0.000,75.600,0.200,10.300,39.0,1.000,106.80,134.4,0.066,3.000,156.500
2393,230,230,corn flakes,108,0.3,0.098,0.071,0.018,24.6,2.200,1.700,0.8,0.0,0.200,0.9,0.0,0.300,1.300,0.026,0.056,0.5,5.900,0.022,0.5,18.200,0.011,0.011,0.000,0.600,0.083,5.4,2.000,0.06,9.2,30.000,0.077,27.947


## EDA
### (Exploratory Data Analysis)

In [5]:
# Exploración Inicial de los Datos
## Verificacion de los nombres de las columnas y sus tipos de datos
print(data_compilada.dtypes)

Unnamed: 0.1              int64
Unnamed: 0                int64
food                     object
Caloric Value             int64
Fat                     float64
Saturated Fats          float64
Monounsaturated Fats    float64
Polyunsaturated Fats    float64
Carbohydrates           float64
Sugars                  float64
Protein                 float64
Dietary Fiber           float64
Cholesterol             float64
Sodium                  float64
Water                   float64
Vitamin A               float64
Vitamin B1              float64
Vitamin B11             float64
Vitamin B12             float64
Vitamin B2              float64
Vitamin B3              float64
Vitamin B5              float64
Vitamin B6              float64
Vitamin C               float64
Vitamin D               float64
Vitamin E               float64
Vitamin K               float64
Calcium                 float64
Copper                  float64
Iron                    float64
Magnesium               float64
Manganes

In [6]:
# Se eliminan las primeras dos filas ya que no aportan informacion mas que el previo indice
data_compilada = data_compilada.drop(data_compilada.columns[[0, 1]], axis=1)
data_compilada

,food,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,Protein,Dietary Fiber,Cholesterol,Sodium,Water,Vitamin A,Vitamin B1,Vitamin B11,Vitamin B12,Vitamin B2,Vitamin B3,Vitamin B5,Vitamin B6,Vitamin C,Vitamin D,Vitamin E,Vitamin K,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
0,eggnog,224,10.6,6.600,3.300,0.500,20.4,20.400,11.600,0.0,149.9,0.100,209.7,0.1,0.074,0.020,0.006,0.500,0.3,1.100,0.100,3.8,0.055,0.500,0.011,330.200,0.051,0.500,48.3,0.024,276.90,419.1,0.094,1.200,377.200
1,beer light,96,0.0,0.000,0.000,0.000,5.4,0.300,0.800,0.0,0.0,0.095,313.1,0.0,0.083,0.079,0.000,0.069,1.3,0.067,0.100,0.0,0.000,0.000,0.000,13.200,0.095,0.014,16.5,0.094,39.60,69.3,0.077,0.044,19.456
2,beer budweiser,12,0.0,0.000,0.000,0.000,0.9,0.000,0.100,0.0,0.0,0.095,27.4,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,1.200,0.095,0.000,2.1,0.038,3.80,9.7,0.000,0.000,2.200
3,weizenbier erdinger,220,18.0,13.000,1.000,0.000,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.000,0.000,18.000
4,beer light budweiser,9,0.0,0.000,0.000,0.000,0.4,0.000,0.066,0.0,0.0,0.016,28.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.900,0.088,0.000,2.1,0.007,3.20,7.7,0.000,0.000,1.320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,muesli master crumble,124,2.4,1.000,19.800,8.500,4.5,0.042,2.400,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.000,0.000,9.300
2391,bran flakes,131,0.8,0.200,0.100,0.500,32.2,7.400,4.000,7.3,0.0,0.200,1.4,0.0,0.300,0.500,0.400,0.011,0.6,6.700,0.300,0.7,0.000,0.090,0.300,0.025,17.600,0.300,11.2,91.600,1.40,179.6,213.200,0.083,45.358
2392,nut cereal,245,5.9,1.000,3.900,0.500,46.2,17.200,4.000,3.6,0.0,0.300,2.6,0.2,0.500,0.300,0.000,0.600,6.5,0.300,0.700,10.7,0.002,0.000,0.000,75.600,0.200,10.300,39.0,1.000,106.80,134.4,0.066,3.000,156.500
2393,corn flakes,108,0.3,0.098,0.071,0.018,24.6,2.200,1.700,0.8,0.0,0.200,0.9,0.0,0.300,1.300,0.026,0.056,0.5,5.900,0.022,0.5,18.200,0.011,0.011,0.000,0.600,0.083,5.4,2.000,0.06,9.2,30.000,0.077,27.947


In [7]:
data_compilada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395 entries, 0 to 2394
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   food                  2395 non-null   object 
 1   Caloric Value         2395 non-null   int64  
 2   Fat                   2395 non-null   float64
 3   Saturated Fats        2395 non-null   float64
 4   Monounsaturated Fats  2395 non-null   float64
 5   Polyunsaturated Fats  2395 non-null   float64
 6   Carbohydrates         2395 non-null   float64
 7   Sugars                2395 non-null   float64
 8   Protein               2395 non-null   float64
 9   Dietary Fiber         2395 non-null   float64
 10  Cholesterol           2395 non-null   float64
 11  Sodium                2395 non-null   float64
 12  Water                 2395 non-null   float64
 13  Vitamin A             2395 non-null   float64
 14  Vitamin B1            2395 non-null   float64
 15  Vitamin B11          

### Limpieza de nombres de Columnas
- Limpieza de los nombres de las columnas, se creo una funcion que elimana los espacios en blanco al principio y al final, cambia el texto de mayusculas a minusculas, sustituye espacios en blanco por '_', elimina todos los acentos del


In [8]:
def limpiar_columnas(nombre):
    nombre = nombre.strip() #Elimina espacios en blanco al principio y al final
    nombre = nombre.replace(' ','_') #Reemplaza espacios en blanco con guiones bajos
    nombre = unidecode(nombre) #Elimina todos los acentos
    nombre = nombre.lower() #Transforma en minuscula todos los nombres de series
    return nombre

data_compilada.columns = data_compilada.columns.map(limpiar_columnas)
data_compilada = data_compilada.copy()
df_test = data_compilada.copy()
data_compilada

,food,caloric_value,fat,saturated_fats,monounsaturated_fats,polyunsaturated_fats,carbohydrates,sugars,protein,dietary_fiber,cholesterol,sodium,water,vitamin_a,vitamin_b1,vitamin_b11,vitamin_b12,vitamin_b2,vitamin_b3,vitamin_b5,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,vitamin_k,calcium,copper,iron,magnesium,manganese,phosphorus,potassium,selenium,zinc,nutrition_density
0,eggnog,224,10.6,6.600,3.300,0.500,20.4,20.400,11.600,0.0,149.9,0.100,209.7,0.1,0.074,0.020,0.006,0.500,0.3,1.100,0.100,3.8,0.055,0.500,0.011,330.200,0.051,0.500,48.3,0.024,276.90,419.1,0.094,1.200,377.200
1,beer light,96,0.0,0.000,0.000,0.000,5.4,0.300,0.800,0.0,0.0,0.095,313.1,0.0,0.083,0.079,0.000,0.069,1.3,0.067,0.100,0.0,0.000,0.000,0.000,13.200,0.095,0.014,16.5,0.094,39.60,69.3,0.077,0.044,19.456
2,beer budweiser,12,0.0,0.000,0.000,0.000,0.9,0.000,0.100,0.0,0.0,0.095,27.4,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,1.200,0.095,0.000,2.1,0.038,3.80,9.7,0.000,0.000,2.200
3,weizenbier erdinger,220,18.0,13.000,1.000,0.000,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.000,0.000,18.000
4,beer light budweiser,9,0.0,0.000,0.000,0.000,0.4,0.000,0.066,0.0,0.0,0.016,28.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.900,0.088,0.000,2.1,0.007,3.20,7.7,0.000,0.000,1.320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,muesli master crumble,124,2.4,1.000,19.800,8.500,4.5,0.042,2.400,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.000,0.000,9.300
2391,bran flakes,131,0.8,0.200,0.100,0.500,32.2,7.400,4.000,7.3,0.0,0.200,1.4,0.0,0.300,0.500,0.400,0.011,0.6,6.700,0.300,0.7,0.000,0.090,0.300,0.025,17.600,0.300,11.2,91.600,1.40,179.6,213.200,0.083,45.358
2392,nut cereal,245,5.9,1.000,3.900,0.500,46.2,17.200,4.000,3.6,0.0,0.300,2.6,0.2,0.500,0.300,0.000,0.600,6.5,0.300,0.700,10.7,0.002,0.000,0.000,75.600,0.200,10.300,39.0,1.000,106.80,134.4,0.066,3.000,156.500
2393,corn flakes,108,0.3,0.098,0.071,0.018,24.6,2.200,1.700,0.8,0.0,0.200,0.9,0.0,0.300,1.300,0.026,0.056,0.5,5.900,0.022,0.5,18.200,0.011,0.011,0.000,0.600,0.083,5.4,2.000,0.06,9.2,30.000,0.077,27.947


### Traduccion de Columna food
La aplicación es en español y los resultados deben ser traducidos

In [9]:
# Crear el objeto traductor
traductor = GoogleTranslator(source='en', target='es')

# Función para traducir con manejo de errores
def traducir_texto(text):
        translated_text = traductor.translate(text)
        return translated_text


# Traducir la columna 'food'
data_compilada['food'] = data_compilada['food'].apply(traducir_texto)

print(data_compilada)



                             food  caloric_value   fat  saturated_fats  \
0                 Ponche de huevo            224  10.6           6.600   
1                  cerveza ligera             96   0.0           0.000   
2               cerveza budweiser             12   0.0           0.000   
3     cerveza weizenbier erdinger            220  18.0          13.000   
4        cerveza budweiser ligera              9   0.0           0.000   
...                           ...            ...   ...             ...   
2390    Crumble maestro de muesli            124   2.4           1.000   
2391          hojuelas de salvado            131   0.8           0.200   
2392             cereal de nueces            245   5.9           1.000   
2393                copos de maíz            108   0.3           0.098   
2394          sémola de maíz seca             36   0.2           0.022   

      monounsaturated_fats  polyunsaturated_fats  carbohydrates  sugars  \
0                    3.300          

In [10]:
# Exploración Inicial de los Datos
## Primeras 40 columnas
data_compilada.head(40)

,food,caloric_value,fat,saturated_fats,monounsaturated_fats,polyunsaturated_fats,carbohydrates,sugars,protein,dietary_fiber,cholesterol,sodium,water,vitamin_a,vitamin_b1,vitamin_b11,vitamin_b12,vitamin_b2,vitamin_b3,vitamin_b5,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,vitamin_k,calcium,copper,iron,magnesium,manganese,phosphorus,potassium,selenium,zinc,nutrition_density
0,Ponche de huevo,224,10.600,6.600,3.300,0.500,20.4,20.400,11.600,0.000,149.9,0.100,209.7,0.1,0.074,0.020,0.006,0.500,0.300,1.100,0.100,3.800,0.055,0.500,0.011,330.20,0.051,0.500,48.3,0.024,276.900,419.1,0.094,1.200,377.200
1,cerveza ligera,96,0.000,0.000,0.000,0.000,5.4,0.300,0.800,0.000,0.0,0.095,313.1,0.0,0.083,0.079,0.000,0.069,1.300,0.067,0.100,0.000,0.000,0.000,0.000,13.20,0.095,0.014,16.5,0.094,39.600,69.3,0.077,0.044,19.456
2,cerveza budweiser,12,0.000,0.000,0.000,0.000,0.9,0.000,0.100,0.000,0.0,0.095,27.4,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.20,0.095,0.000,2.1,0.038,3.800,9.7,0.000,0.000,2.200
3,cerveza weizenbier erdinger,220,18.000,13.000,1.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000,18.000
4,cerveza budweiser ligera,9,0.000,0.000,0.000,0.000,0.4,0.000,0.066,0.000,0.0,0.016,28.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.90,0.088,0.000,2.1,0.007,3.200,7.7,0.000,0.000,1.320
5,cerveza,142,0.000,0.000,0.000,0.000,11.7,0.000,1.500,0.000,0.0,0.067,303.5,0.0,0.093,0.062,0.000,0.017,1.700,0.100,0.200,0.000,0.000,0.000,0.000,13.20,0.096,0.031,19.8,0.085,46.200,89.1,0.087,0.001,26.436
6,cerveza de raíz,202,0.000,0.000,0.000,0.000,52.3,52.300,0.000,0.000,0.0,0.068,440.2,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,24.700,0.037,0.2,4.900,0.059,0.0,4.900,0.095,52.334
7,mezcla de whisky sour,26,0.072,0.069,0.034,0.012,6.3,6.300,0.051,0.000,0.0,0.073,23.1,0.0,0.089,0.000,0.000,0.070,0.000,0.032,0.000,0.800,0.000,0.000,0.000,0.60,0.000,0.047,0.3,0.000,1.800,8.3,0.015,0.080,7.914
8,Whisky agrio,158,0.000,0.018,0.092,0.024,13.9,0.000,0.000,0.000,0.0,0.067,77.0,0.0,0.035,0.000,0.000,0.092,0.003,0.050,0.000,1.700,0.000,0.000,0.000,1.10,0.019,0.047,1.1,0.010,6.400,5.3,0.030,0.069,16.717
9,amanecer de tequila,232,0.200,0.000,0.000,0.000,23.8,0.000,0.600,0.000,0.0,0.100,166.3,0.0,0.085,0.010,0.000,0.015,0.400,0.200,0.100,40.700,0.000,0.000,0.000,0.00,0.010,0.038,14.8,0.046,21.100,21.1,0.000,1.300,65.357


In [11]:
## Ultimas 40 filas
data_compilada.tail(40)

,food,caloric_value,fat,saturated_fats,monounsaturated_fats,polyunsaturated_fats,carbohydrates,sugars,protein,dietary_fiber,cholesterol,sodium,water,vitamin_a,vitamin_b1,vitamin_b11,vitamin_b12,vitamin_b2,vitamin_b3,vitamin_b5,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,vitamin_k,calcium,copper,iron,magnesium,manganese,phosphorus,potassium,selenium,zinc,nutrition_density
2355,semillas de girasol tostadas,829,76.100,8.000,14.500,50.300,27.6,0.000,23.1,15.4,0.0,0.800,1.300,0.000,0.400,0.300,0.000,0.400,5.600,9.500,1.100,1.900,0.000,0.000,0.000,76.400,2.500,9.100,172.9,2.800,1551.70,657.9,0.066,7.100,229.600
2356,semillas de sisymbrium,235,3.400,0.700,1.100,1.500,43.1,0.000,9.0,0.0,0.0,0.047,4.500,0.000,0.100,0.072,0.000,0.300,12.500,0.700,0.600,22.700,0.000,0.000,0.000,1208.400,0.052,0.098,232.4,1.100,4.40,1576.2,0.000,0.200,1286.643
2357,semillas del árbol del pan secas,587,2.700,0.700,0.300,1.400,127.0,0.000,13.8,23.8,0.0,0.091,10.400,0.000,0.040,0.200,0.000,0.200,3.400,3.000,1.100,74.600,0.000,0.000,0.000,150.400,3.900,7.400,184.0,0.500,284.80,3217.6,0.011,3.100,399.700
2358,nueces pecanas crudas,753,78.400,6.700,44.500,23.600,15.1,4.300,10.0,10.5,0.0,0.000,3.800,0.000,0.700,0.019,0.000,0.100,1.300,0.900,0.200,1.200,0.000,1.500,0.033,76.300,1.300,2.800,131.9,4.900,301.90,446.9,0.039,4.900,194.300
2359,semillas de chia secas,138,8.700,0.900,0.700,6.700,11.9,0.000,4.7,9.7,0.0,0.047,1.600,0.000,0.200,0.098,0.000,0.080,2.500,0.000,0.000,0.500,0.000,0.100,0.000,178.600,0.300,2.200,94.8,0.800,243.40,115.2,0.035,1.300,216.300
2360,castañas japonesas asadas,57,0.200,0.080,0.100,0.094,12.8,0.000,0.8,0.0,0.0,0.079,14.100,0.000,0.100,0.022,0.000,0.000,0.200,0.100,0.100,7.900,0.000,0.000,0.000,9.900,0.200,0.600,18.1,0.600,26.30,120.8,0.000,0.400,32.200
2361,semillas de sésamo tostadas,45,3.800,0.500,1.500,1.700,2.1,0.000,1.4,1.1,0.0,0.070,0.300,0.000,0.084,0.075,0.000,0.022,0.400,0.074,0.002,0.000,0.000,0.000,0.000,79.100,0.200,1.200,28.5,0.200,51.00,38.0,0.030,0.600,88.700
2362,anacardos tostados,174,14.300,2.500,7.800,2.600,9.0,1.500,5.1,1.0,0.0,0.091,0.700,0.000,0.100,0.080,0.000,0.025,0.500,0.300,0.056,0.071,0.000,0.300,0.041,12.900,0.600,1.800,81.9,0.500,159.30,189.6,0.020,1.600,44.120
2363,altramuces crudos,668,17.500,2.100,7.100,4.400,72.7,0.000,65.1,34.0,0.0,0.009,18.800,0.000,1.200,0.600,0.000,0.400,3.900,1.400,0.600,8.600,0.000,0.000,0.000,316.800,1.800,7.800,356.4,4.300,792.00,1823.4,0.090,8.600,522.500
2364,semillas de mostaza molidas,10,0.700,0.012,0.500,0.200,0.6,0.100,0.5,0.2,0.0,0.017,0.100,0.000,0.072,0.093,0.000,0.020,0.093,0.018,0.000,0.100,0.000,0.100,0.086,5.300,0.055,0.200,7.4,0.078,16.60,14.8,0.003,0.100,7.600


In [12]:
## Registros Random
data_compilada.iloc[40:-40].sample(40)

,food,caloric_value,fat,saturated_fats,monounsaturated_fats,polyunsaturated_fats,carbohydrates,sugars,protein,dietary_fiber,cholesterol,sodium,water,vitamin_a,vitamin_b1,vitamin_b11,vitamin_b12,vitamin_b2,vitamin_b3,vitamin_b5,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,vitamin_k,calcium,copper,iron,magnesium,manganese,phosphorus,potassium,selenium,zinc,nutrition_density
2113,batata enlatada,568,1.300,0.300,0.080,0.500,133.5,98.300,6.300,16.000,0.0,0.300,493.700,0.000,0.200,0.074,0.000,0.300,2.900,2.100,0.300,67.000,0.000,6.000,0.098,95.700,0.800,5.100,82.900,3.200,172.300,1180.3,0.037,1.200,324.900
588,Delicia de clara de huevo de McDonald's,250,16.000,4.000,18.000,4.000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,16.000
610,panqueques de mcdonalds,113,2.900,0.600,0.600,1.500,19.0,4.400,3.000,0.700,6.4,0.200,23.900,0.000,0.100,0.088,0.000,0.100,1.100,0.400,0.005,0.000,0.000,0.300,0.000,41.700,0.054,0.900,9.400,0.100,129.000,85.8,0.000,0.200,68.200
1885,Tomatillos,42,1.300,0.200,0.200,0.600,7.7,5.200,1.300,2.500,0.0,0.019,121.000,0.000,0.088,0.093,0.000,0.099,2.400,0.200,0.074,15.400,0.000,0.500,0.033,9.200,0.100,0.800,26.400,0.200,51.500,353.8,0.055,0.300,38.200
1066,mostaza,3,0.200,0.021,0.100,0.086,0.3,0.048,0.200,0.200,0.0,0.015,4.200,0.096,0.026,0.006,0.097,0.000,0.011,0.040,0.013,0.015,0.030,0.000,0.086,0.012,3.200,0.086,0.096,2.400,0.075,5.4,7.600,0.064,1.046
1910,cebolla dulce,106,0.300,0.000,0.000,0.000,25.0,16.600,2.600,3.000,0.0,0.063,302.000,0.000,0.000,0.100,0.047,0.000,0.062,0.400,0.300,0.400,15.900,0.000,0.038,0.033,66.200,0.200,0.900,29.800,0.300,89.4,393.900,0.067,31.600
2003,guisantes partidos crudos,672,2.300,0.300,0.500,1.000,118.9,15.800,48.400,50.200,0.0,0.075,22.200,0.000,1.400,0.500,0.000,0.400,5.700,3.500,0.300,3.500,0.000,0.200,0.037,108.400,1.700,8.700,226.600,2.700,721.000,1932.6,0.037,5.900,340.400
960,ciruelas pasas enlatadas,246,0.500,0.072,0.300,0.100,65.1,0.000,2.000,8.900,0.0,0.044,165.400,0.000,0.044,0.000,0.000,0.300,2.000,0.200,0.500,6.600,0.000,0.000,0.000,39.800,0.300,1.000,35.100,0.200,60.800,528.8,0.000,0.400,123.900
1832,soja cocida,296,15.400,2.200,3.400,8.700,17.1,5.200,28.600,10.300,0.0,0.075,107.600,0.000,0.000,0.300,0.022,0.000,0.500,0.700,0.300,0.400,2.900,0.000,0.600,0.099,175.400,0.700,8.800,147.900,1.400,421.4,885.800,0.064,72.517
1568,pan de salvado de arroz,66,1.200,0.200,0.400,0.500,11.7,1.300,2.400,1.300,0.0,0.081,11.100,0.000,0.200,0.063,0.000,0.051,1.800,0.200,0.092,0.000,0.000,0.200,0.019,18.600,0.039,1.000,21.600,0.400,48.100,58.1,0.061,0.400,36.200


In [13]:
### Faltan Graficas

In [14]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Ruta del archivo en el directorio actual
ruta_archivo = os.path.join(directorio_actual, "data_compilada.csv")

# Guardar el dataframe en un archivo CSV, sustituyendo si ya existe
data_compilada.to_csv(ruta_archivo, index=False)

print(f"Archivo guardado en {ruta_archivo}")

Archivo guardado en C:\Users\HP\Desktop\FORMACION\HackaBoss Data Scienct AI\data_compilada.csv


## Machne Learning
Se calculan las distancias entre cada elemento con el modelo NearestNeighbour para luego realizar la recomendacion con el modelo k-nn

In [15]:
## NearestNeigbours
X = data_compilada.drop(['food', 'saturated_fats',
       'monounsaturated_fats', 'polyunsaturated_fats',
       'dietary_fiber', 'cholesterol', 'sodium', 'water',
       'vitamin_a', 'vitamin_b1', 'vitamin_b11', 'vitamin_b12', 'vitamin_b2',
       'vitamin_b3', 'vitamin_b5', 'vitamin_b6', 'vitamin_c', 'vitamin_d',
       'vitamin_e', 'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium',
       'manganese', 'phosphorus', 'potassium', 'selenium', 'zinc',
       'nutrition_density'], axis=1).values
food_names = data_compilada['food'].values

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Entrenar el Modelo
nneigh = NearestNeighbors(metric='cosine', algorithm='brute')
nneigh.fit(X_scaled)

# Guardar el Modelo
joblib.dump(nneigh, 'nearest_neighbors_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

### Realizar recomendaciones usando el model KNN

In [16]:

# Cargar el modelo y el scaler
nneigh = joblib.load('nearest_neighbors_model.pkl')
scaler = joblib.load('scaler.pkl')

# Valores de entrada (caloric_value', 'fat',  'carbohydrates', 'protein', 'sugars')
input_values = np.array([[10, 5, 20, 3, 23]])

# Normalizar los valores de entrada
input_scaled = scaler.transform(input_values)

# Encontrar los 5 vecinos más cercanos
distances, indices = nneigh.kneighbors(input_scaled, n_neighbors=5)

# Obtener los nombres de los alimentos recomendados
recommended_foods = food_names[indices[0]]

print("Alimentos recomendados:", recommended_foods)


Alimentos recomendados: ['sopa de pollo con verduras' 'requesón sin grasa'
 'concentrado de proteínas de soja'
 'harina de semilla de algodón baja en grasa'
 'harina de sésamo baja en grasa']


In [17]:
### caloric_value', 'fat',  'carbohydrates', 'protein', 'sugars',